In [2]:
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
#from pNbody import*

from concaveDelaunayRefinement import*
from toolMockImage import*

import pickle
import yaml
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

In [16]:
from toolMockImage import*
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import pickle 
import yaml

with open("paraImage.yaml") as stream:
    try:
        para = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

name = para["SimulationName"]
trans = para["translation"]
rot = para["rotation"]
triName = para["triSaveName"]
NeighborsName = para["NeighborsSaveName"]
InitialFrame = para["InitialFrameName"]
triangleFolder = para["triangleFolder"]
axe1 = para["axe1"]
axe2 = para["axe2"]
Nmin = para["Nmin"]
Rmax = para["Rmax"]
halfBoxSize = para["halfBoxSize"]

In [15]:
name = '/srv/astro/projects/clastro/revaz/ARRAKIHS/SWIFT_UFDS_TidalStripping/Sims/DW103/snap/snapshot_0066_stars.hdf5'
nb = Nbody(name,ftype='swift')

nb.translate(trans)
print(nb)

In [19]:
def download_simulation(name,trans,rot):
    """
    def download_simulation(name,trans,rot):
    Download un fichier hdf5 contenant une snapshot de Nbody simulation.
    La function utilise pNbody et la simulation doit provenir du code SWIFT.
    - name : string du path au ficheir hdf5
    - trans : vecteur [x,y,z] de la translation de la simulation avec pNbody, nb = nb.translate(trans) (le code a besoin que les particules soit centrer en (0,0,0))
    - rota : angle de rotation de pNbody, nb = nb.rotate(angle=rot)
    """


    nb = Nbody(name,ftype='swift')

    nb.translate(trans)
    #nb = nb.rotate(angle=rot)
    return nb


In [20]:
nb0 = download_simulation(name,trans,rot)

In [21]:
nb0.pos

array([[-221.4454   ,  -92.253265 ,  -21.174255 ],
       [-173.30063  ,    5.2429504,   59.71399  ],
       [ -96.02725  ,   -4.821228 , -110.23126  ],
       ...,
       [  96.16742  ,   32.76367  ,  111.48193  ],
       [ 134.60632  ,   95.389465 ,  -68.747375 ],
       [ 179.64587  ,  -23.69696  ,  -92.77368  ]], dtype=float32)

In [22]:

std_list = compute_std(nb0)
nb0_scaled = scale_nb(nb0,std_list)

# test de PartialImage.py

In [1]:
print('Library')

import numpy as np
print('numpy')
from scipy.spatial import Delaunay
print("Delaunay")
import matplotlib.pyplot as plt
print("matplotlib.pyplot")
from pNbody import*
print("pNbody")

from concaveDelaunayRefinement import*
print("concaveDelaunayRefinement")

from toolMockImage import*
print("toolMockImage")

import pickle
print("pickle")
import yaml
print("yaml")
import matplotlib
print("matplolib")
matplotlib.rcParams.update({'font.size': 14})

import sys
print("sys")


Library
numpy
Delaunay
matplotlib.pyplot


--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: lesta02
--------------------------------------------------------------------------


pNbody
concaveDelaunayRefinement
toolMockImage
pickle
yaml
matplolib
sys


In [7]:

print("Partial Image start")

with open("paraImage.yaml") as stream:
    try:
        para = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

name = para["SimulationName"]
trans = para["translation"]
triName = para["triSaveName"]
NeighborsName = para["NeighborsSaveName"]
InitialFrame = para["InitialFrameName"]
triangleFolder = para["triangleFolder"]
axe1 = para["axe1"]
axe2 = para["axe2"]
Nmin = para["Nmin"]
Rmax = para["Rmax"]
halfBoxSize = para["halfBoxSize"]


print('scaling data')
nb0 = download_simulation(InitialFrame,trans)
std_list = compute_std(nb0)
nb0_scaled = scale_nb(nb0,std_list)

nb = download_simulation(name,trans)
nb_scaled = scale_nb(nb,std_list)

nb_scaled_tList = [nb0_scaled,nb_scaled]

file_path = NeighborsName
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)

print("The variable 'Neighbors' has been loaded successfully.")

neighbors = loaded_data

file_path = triName
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)

print("The variable 'Tri' has been loaded successfully.")

tri0 = loaded_data

filePara = 1#int(sys.argv[1])

parameter_a_list = []
#for ii in range(10):
#    parameter_a_list.append(filePara*10+ii)
parameter_a_list.append(filePara)
print(f'parameter_a_list ={parameter_a_list}')


Partial Image start
scaling data
The variable 'Neighbors' has been loaded successfully.
The variable 'Tri' has been loaded successfully.
parameter_a_list =[1]


### code retirer a toolMockImage.py

In [13]:
def Scale_triangle_2d(vertex,std_List,ax1,ax2,halfBoxSize):
    """
    def Scale_triangle_2d(vertex,std_List,ax1,ax2,halfBoxSize):
    
    this function scale the triangle's vertex to be projected. First std_list is use again to scale each coordinate to his true value. Then halfBoxSize is used such as all particle between [-halfBoxSize,halfBoxSize] is between [-1,1] to be prjected by pyOpenGl.
    The image will include particle with coordinate in [-halfBoxSize,halfBoxSize].
    input:
        - vertex : list of vertex of the projected triangle. List of 2 dimensional array
        - std_List : List [std_x,std_y,std_z,std_vx,std_vy,std_vz] compute by compute_std(nb)
        - ax1 : axe of x coordinate of the image
        - ax2 : axe of y coordinate of the image
        - halfBoxSize : the half of the boxSize of the image (in Unit of the simulation)
    return:
        - List of vertex scale by halfBoxSize from the original value in the simulation
    """
    #print("Scale_triangle_2d")
    for ii in range(len(vertex)):
        vertex[ii][0] = vertex[ii][0]/halfBoxSize*std_List[ax1]
        vertex[ii][1] = vertex[ii][1]/halfBoxSize*std_List[ax2]
    return vertex

def projection_6d_to_2d(tri,id_list,ax1,ax2):
    """
    def projection_6d_to_2d(tri,id_list,ax1,ax2):
    
        fonction to compute the projection

    input: 
        - tri : Delaunay triangulation of the image frame
        - id_list : id_list of the simplices in the concave refinement obtain with concaveDelaunayRefinement.py
        - ax1 : axe of x coordinate of the image
        - ax2 : axe of y coordinate of the image
    return:
        - tri2d_list: List of Delaunay triangulation in two dimension. The triangle of the triangulation are the triangle to display
        - for each triangle, a parameter alpha (matrix of float) representing it's intensity (abitrary unit). (it's a matrix alpha[trianglulation][triangle of the triangulation])
    """
    print("projection_6d_to_2d")
    tri2d_list = []
    alpha_list = []
    jj=0
    n=len(id_list)
    for ii in id_list:
        jj=jj+1
        V = Tetrahedron6DVolume(tri.points[tri.simplices[ii]])
        w2d = simplex_projection2d(tri.points[tri.simplices[ii]],ax1,ax2)
        tri2d = Delaunay(w2d)
        V2d_ListSimplices = delaunay_volume(tri2d)
        V2d = np.sum(V2d_ListSimplices)
        for ii in range(len(V2d_ListSimplices)):
            V2d_ListSimplices[ii] = V/V2d
        alpha = V2d_ListSimplices
        tri2d_list.append(tri2d)
        alpha_list.append(alpha)
    return tri2d_list,alpha_list

def VertexList2d(tri,std_List,id_list,ax1,ax2,halfBoxSize):
    """
    input: 
        - tri : Delaunay triangulation of the image frame
        - id_list : id_list of the simplices in the concave refinement obtain with concaveDelaunayRefinement.py
        - ax1 : axe of x coordinate of the image
        - ax2 : axe of y coordinate of the image
        - halfBoxSize: the image will contain particle in [-halfBoxSize,halfBoxSize]
    return:
        - triangle_List: List of triangle (three two-dimensonal array) to display
        - for each triangle, a parameter alpha (float) representing it's intensity (abitrary unit)
    """
    print("vertexList2d")
    triangle_List = []
    alpha = []
    tri2d_list,alpha_list = projection_6d_to_2d(tri,id_list,ax1,ax2)
    for ii in range(len(tri2d_list)):
        for jj in range(len(tri2d_list[ii].simplices)):
            vertices = tri2d_list[ii].points[tri2d_list[ii].simplices[jj]]
            triangle_List = [*triangle_List,*Scale_triangle_2d(vertices,std_List,ax1,ax2,halfBoxSize)]
            alpha.append(alpha_list[ii][jj])
    return triangle_List,alpha

In [14]:

for parameter_a in parameter_a_list:
    cell_id_list = []
    triangle_List = []
    opacity_list = []
    data = []
    for ii in range(len(nb_scaled_tList[0].num)):
        if ii==25:#ii%100==parameter_a:
            cell_id_list.append(ii)

    n = len(cell_id_list)
    jj=0
    print(f'DEBBUG :: tri-file parameter {parameter_a}')
    
    for p_id in cell_id_list:
        jj=jj+1
        print(f'DEBBUG :: iteration {jj} of {len(cell_id_list)}')
        print(f'DEBBUG :: cell creation')
        w_scale_tList = create_w_tList_cell(nb_scaled_tList,neighbors,p_id,Rmax=Rmax)
        flag = 0
        if len(w_scale_tList[0])<Nmin:
            flag=1
        else:
            print('flag=0')
            print('DEBUG : Concave Delaunay Algorithm')
            index_conserved_simplex_tList,tri_tList,flag_tList = Volume_evolution_Liouville_condition_concave_delaunay_refinements(w_scale_tList,error_max=5,verbos=False)
            flag = flag_tList[-1]
            

            index_List = index_conserved_simplex_tList[-1]
            tri = tri_tList[-1]
            V_cons = delaunay_volume_6D(tri_tList[0])

            if flag==0:
                print(0)
                tmp,opa = VertexList2d(tri,std_list,index_List,axe1,axe2,halfBoxSize)
                print(1)
                opa = opa/V_cons
                triangle_List = [*triangle_List,*tmp]
                opacity_list = [*opacity_list,*opa]
        if flag==1:
            # code to add light in an other way
            print('flag=1')
    """
    print(f'number of triangle :: {len(triangle_List)}')
    name = f'Partial_Image_{parameter_a}'

    
    for ii in range(len(triangle_List)):
        data.append([triangle_List[ii][0],triangle_List[ii][1],opacity_list[ii//3]])


    file_path_data = f'{triangleFolder}/{name}_data.pickle'
    with open(file_path_data, 'wb') as file:
        # Serialize and write the variable to the file
        pickle.dump(data, file)

    print(f'The variable "data" has been saved successfully. \n File name :: {file_path_data}')"""

DEBBUG :: tri-file parameter 1
DEBBUG :: iteration 1 of 1
DEBBUG :: cell creation
flag=0
DEBUG : Concave Delaunay Algorithm
DV/V = 1.671577891444908%
0
vertexList2d
projection_6d_to_2d


KeyboardInterrupt: 